# find git repo
[here](https://github.com/despresj/ece_colab)

In [1]:
!nvidia-smi -L

zsh:1: command not found: nvidia-smi


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

In [4]:
df = pd.read_csv(f"{proj_dir}data_clean/clean.csv").to_numpy()

In [5]:
generator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(98, activation="sigmoid")
])

2022-02-21 08:16:57.615774: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
discriminator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid") # not correct output dim
])

In [7]:
gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [8]:
codings_size = 98
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(df).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [27]:
# Although this runs it still has bugs
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1} of {n_epochs}")
        for _ in range(1):
            X_batch = np.random.permutation(df)[:100]
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=(100,98))
            generated_data = generator(noise)
            X_fake_and_real = tf.concat([generated_data, X_batch], axis=0)

            y1 = tf.constant([[0.]] * noise.shape[0] + [[1.]] * X_batch.shape[0])
            y1 = np.reshape(y1, (200, 1))

            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)

In [28]:
train_gan(gan, df, batch_size, codings_size, n_epochs=1)

Epoch 1 of 1


In [ ]:
import os
import re
gans_saved = os.listdir(f"{proj_dir}saved_models/")
model_number = [int(re.sub("[^\d\.]", "", x)) for x in gans_saved]
last_model = max(model_number)

In [ ]:
generator.save(f"{proj_dir}saved_models/gan{last_model+1}")

In [29]:
noise = tf.random.normal(shape=[batch_size, codings_size])
generated_data = generator(noise)
generated_data

<tf.Tensor: shape=(32, 98), dtype=float32, numpy=
array([[0.5505153 , 0.80438536, 0.91929984, ..., 0.38824248, 0.70006466,
        0.8162012 ],
       [0.21477133, 0.3638113 , 0.32969555, ..., 0.41733658, 0.509904  ,
        0.8086928 ],
       [0.36741313, 0.5956406 , 0.56489897, ..., 0.7207143 , 0.30245823,
        0.6437542 ],
       ...,
       [0.78093016, 0.6121604 , 0.87179005, ..., 0.56019   , 0.7248996 ,
        0.4192207 ],
       [0.18149596, 0.59600335, 0.83198065, ..., 0.21240863, 0.6818019 ,
        0.44930172],
       [0.74663967, 0.32261834, 0.61028486, ..., 0.5345375 , 0.29804528,
        0.30619386]], dtype=float32)>

In [ ]:
!git config --global user.email "despresj@mail.gvsu.edu"
!git config --global user.name "despresj"